### Run One-Sample T-Tests for each subcategory and system

In [39]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [3]:
# Import csv file from path

path = '/Users/Sofia/Desktop/cognitive-project'

df = pd.read_csv(path + '/results.csv')

In [6]:
df.head()

,Subcategories,Random\nSpearman,BUAP\nSpearman,UTD-NB\nSpearman,UTD-SVM\nSpearman,UMD-V0\nSpearman,UMD-V1\nSpearman,UMD-V2\nSpearman,Llama 2 Spearman,Random\nMaxDiff,BUAP\nMaxDiff,UTD-NB\nMaxDiff,UTD-SVM\nMaxDiff,UMD-V0\nMaxDiff,UMD-V1\nMaxDiff,UMD-V2\nMaxDiff,Llama 2 MaxDiff
0,1b CLASS-INCLUSION -- Functional,0.226,0.072,0.098,0.000,0.095,0.333,0.135,0.169,35.4,31.6,35.9,24.8,29.1,38.3,30.6,38.8
1,1c CLASS-INCLUSION -- Singular Collective,-0.142,0.075,0.178,0.109,-0.171,0.160,0.050,-0.065,23.3,31.9,33.3,31.4,20.5,36.7,30.0,29.0
2,1d CLASS-INCLUSION -- Plural Collective,-0.001,-0.088,0.656,0.264,0.023,0.012,0.168,-0.251,31.5,23.6,48.1,37.0,30.6,28.2,33.3,25.0
3,1e CLASS-INCLUSION -- ClassIndividual,0.145,0.196,0.000,0.000,0.233,0.206,0.317,0.082,33.7,38.6,33.1,33.1,36.7,39.2,38.6,26.5
4,2a PART-WHOLE -- Object:Component,-0.240,0.048,0.209,0.240,0.107,0.075,0.093,0.025,16.8,35.5,36.4,42.7,32.7,32.3,33.6,28.6


In [8]:
# Clean up data

# remove new line characters from column names
df.columns = df.columns.str.replace('\n', '')

In [9]:
df.head()

,Subcategories,RandomSpearman,BUAPSpearman,UTD-NBSpearman,UTD-SVMSpearman,UMD-V0Spearman,UMD-V1Spearman,UMD-V2Spearman,Llama 2 Spearman,RandomMaxDiff,BUAPMaxDiff,UTD-NBMaxDiff,UTD-SVMMaxDiff,UMD-V0MaxDiff,UMD-V1MaxDiff,UMD-V2MaxDiff,Llama 2 MaxDiff
0,1b CLASS-INCLUSION -- Functional,0.226,0.072,0.098,0.000,0.095,0.333,0.135,0.169,35.4,31.6,35.9,24.8,29.1,38.3,30.6,38.8
1,1c CLASS-INCLUSION -- Singular Collective,-0.142,0.075,0.178,0.109,-0.171,0.160,0.050,-0.065,23.3,31.9,33.3,31.4,20.5,36.7,30.0,29.0
2,1d CLASS-INCLUSION -- Plural Collective,-0.001,-0.088,0.656,0.264,0.023,0.012,0.168,-0.251,31.5,23.6,48.1,37.0,30.6,28.2,33.3,25.0
3,1e CLASS-INCLUSION -- ClassIndividual,0.145,0.196,0.000,0.000,0.233,0.206,0.317,0.082,33.7,38.6,33.1,33.1,36.7,39.2,38.6,26.5
4,2a PART-WHOLE -- Object:Component,-0.240,0.048,0.209,0.240,0.107,0.075,0.093,0.025,16.8,35.5,36.4,42.7,32.7,32.3,33.6,28.6


In [56]:
def get_N(subcategory):
    if subcategory == '1b':
        N = 41
    elif subcategory == '1c':
        N = 42
    elif subcategory == '1d':
        N = 43
    elif subcategory == '1e':
        N = 33
    elif subcategory == '2a':
        N = 44
    elif subcategory == '2b':
        N = 38
    elif subcategory == '2d':
        N = 39
    elif subcategory == '2e':
        N = 40
    elif subcategory == '2f':
        N = 43
    elif subcategory == '2g':
        N = 43
    elif subcategory == '2i':
        N = 39
    elif subcategory == '2j':
        N = 42
    elif subcategory == '3b':
        N = 43
    elif subcategory == '3d':
        N = 41
    elif subcategory == '3e':
        N = 37
    elif subcategory == '3f':
        N = 45
    elif subcategory == '3g':
        N = 38
    elif subcategory == '3h':
        N = 43
    elif subcategory == '4a':
        N = 43
    elif subcategory == '4b':
        N = 35
    elif subcategory == '4d':
        N = 30
    elif subcategory == '4e':
        N = 45
    elif subcategory == '4f':
        N = 44
    elif subcategory == '4g':
        N = 45
    else:
        N = 43
    return N

In [60]:
# Perform a one sample t-test for each value in a single column
# Store the p-values in a dictionary for each subcategory

# Create an empty dictionary to store p-values
p_values = {}

# Iterate through unique subcategories
for subcategory in df['Subcategories'].unique():
    # print 1st two characters of subcategory
    sub = str(subcategory[:2])
    N = get_N(subcategory)
    p_values[subcategory] = {}
    for col in df.columns[1:9]:
        rho = np.array(df.loc[df['Subcategories'] == subcategory, col])[0]
        rho = float(rho)
        if not np.isnan(rho):
            t_statistic = (rho * np.sqrt(N - 2)) / np.sqrt(1 - rho**2)
            # Calculate the p-value
            degrees_of_freedom = N - 2
            p_value = 2 * (1 - stats.t.cdf(abs(t_statistic), df=degrees_of_freedom))
            p_values[subcategory][col] = p_value
        else:
            p_values[subcategory][col] = np.nan

In [69]:
# Show p-values for each subcategory less than 0.05

buap_cnt = 0
utd_cnt = 0
duluth_cnt = 0
llama_cnt = 0
random_cnt = 0
tot_cnt = 0

# Count how many p-values are less than 0.05 for each system: BUAP, UTD, Duluth
for subcategory in p_values:
    print(subcategory)
    for key, value in p_values[subcategory].items():
        if value < 0.05:
            tot_cnt += 1
            if 'BUAP' in key:
                buap_cnt += 1
            elif 'UTD' in key:
                utd_cnt += 1
            elif 'UMD' in key:
                duluth_cnt += 1
            elif 'Llama' in key:
                llama_cnt += 1
            print(key, value)

# Print the counts
print('Total:', tot_cnt)
print('BUAP:', buap_cnt)
print('UTD:', utd_cnt)
print('Duluth:', duluth_cnt)
print('Llama:', llama_cnt)
print('Random:', random_cnt)

1b CLASS-INCLUSION -- Functional
UMD-V1Spearman 0.02911108575016974
1c CLASS-INCLUSION -- Singular Collective
1d CLASS-INCLUSION -- Plural Collective
UTD-NBSpearman 1.7933802634040319e-06
1e CLASS-INCLUSION -- ClassIndividual
UMD-V2Spearman 0.03833929202307762
2a PART-WHOLE -- Object:Component
2b PART-WHOLE -- Collection:Member
UTD-SVMSpearman 0.02140409653086861
Llama 2 Spearman 0.0002260402277103335
2d PART-WHOLE -- Event:Feature
2e PART-WHOLE -- Activity:Stage
2f PART-WHOLE -- Item:Topological Part
UTD-NBSpearman 1.0073519734810077e-05
Llama 2 Spearman 0.0013592938257156462
2g PART-WHOLE -- Object:Stuff
UTD-NBSpearman 0.002859107924289672
2i PART-WHOLE -- Item:Distinctive Nonpart
2j PART-WHOLE -- Item:Ex-part/Ex-possession
UMD-V0Spearman 0.02390115040868812
UMD-V1Spearman 0.00151195034737861
UMD-V2Spearman 0.011954788033358366
Llama 2 Spearman 0.030684927948060547
3b SIMILAR -- Dimensional Similarity
3d SIMILAR -- Dimensional Naughty
3e SIMILAR -- Conversion
UTD-NBSpearman 2.8962921

In [70]:
# Show p-values for each subcategory less than 0.01

buap_cnt = 0
utd_cnt = 0
duluth_cnt = 0
llama_cnt = 0
random_cnt = 0
tot_cnt = 0

# Count how many p-values are less than 0.05 for each system: BUAP, UTD, Duluth
for subcategory in p_values:
    print(subcategory)
    for key, value in p_values[subcategory].items():
        if value < 0.01:
            tot_cnt += 1
            if 'BUAP' in key:
                buap_cnt += 1
            elif 'UTD' in key:
                utd_cnt += 1
            elif 'UMD' in key:
                duluth_cnt += 1
            elif 'Llama' in key:
                llama_cnt += 1
            print(key, value)

# Print the counts
print('Total:', tot_cnt)
print('BUAP:', buap_cnt)
print('UTD:', utd_cnt)
print('Duluth:', duluth_cnt)
print('Llama:', llama_cnt)
print('Random:', random_cnt)

1b CLASS-INCLUSION -- Functional
1c CLASS-INCLUSION -- Singular Collective
1d CLASS-INCLUSION -- Plural Collective
UTD-NBSpearman 1.7933802634040319e-06
1e CLASS-INCLUSION -- ClassIndividual
2a PART-WHOLE -- Object:Component
2b PART-WHOLE -- Collection:Member
Llama 2 Spearman 0.0002260402277103335
2d PART-WHOLE -- Event:Feature
2e PART-WHOLE -- Activity:Stage
2f PART-WHOLE -- Item:Topological Part
UTD-NBSpearman 1.0073519734810077e-05
Llama 2 Spearman 0.0013592938257156462
2g PART-WHOLE -- Object:Stuff
UTD-NBSpearman 0.002859107924289672
2i PART-WHOLE -- Item:Distinctive Nonpart
2j PART-WHOLE -- Item:Ex-part/Ex-possession
UMD-V1Spearman 0.00151195034737861
3b SIMILAR -- Dimensional Similarity
3d SIMILAR -- Dimensional Naughty
3e SIMILAR -- Conversion
UTD-NBSpearman 2.8962921825970156e-05
3f SIMILAR -- Attribute Similarity
3g SIMILAR -- Coordinates
UTD-NBSpearman 0.0005059611616340387
UMD-V0Spearman 0.0034713886164889907
UMD-V1Spearman 7.629597207858829e-05
UMD-V2Spearman 2.678974164860